In [1]:
import json
import os
import shutil
import subprocess
import tempfile
import glob
import pandas as pd
from functools import reduce
import re
import pandas as pd
import time

intersect_peaks_file_path = '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/ZYMO15-round1_allpeaks.bed' #/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/ZYMO15_Allpeaks.bed
 # '/home/joaquin/projects/methylation/data/data_concentraciones/ZYMO/15/1ng', #/ZYMO71ngsorted.bam
folders = [
   '/home/joaquin/projects/methylation/data/data_concentraciones/ZYMO/15/1ng',
    '/home/joaquin/projects/methylation/data/data_concentraciones/ZYMO/15/5ng', #/ZYMO75ngsorted.bam
    '/home/joaquin/projects/methylation/data/data_concentraciones/ZYMO/15/10ng', #/ZYMO710ngsorted.bam
    '/home/joaquin/projects/methylation/data/data_concentraciones/ZYMO/15/25ng', #/ZYMO725ngsorted.bam
    '/home/joaquin/projects/methylation/data/data_concentraciones/ZYMO/15/50ng', #/ZYMO750ngsorted.bam
    '/home/joaquin/projects/methylation/data/data_concentraciones/ZYMO/15/Input' #/ZYMO7Input.bam

]

In [3]:
def performIntersect(folder, intersectFile):
    
    sortedBamFile = glob.glob(f'{folder}/*orted.bam')
    
    if len(sortedBamFile) != 1:
        return print(folder, ' has a problem selecting File')
    else:
        sortedBamFile = sortedBamFile[0]
    
    outputFile = intersectFile.strip().split('/')[-1][:-4]+'_'+sortedBamFile.split('/')[-1][:-10]+'.bed'
    outputFilePath = os.path.join(folder,outputFile)

    print(outputFilePath)
    print(outputFilePath)
    
    if os.path.isfile(outputFilePath):
        os.remove(outputFilePath)
    if not os.path.isfile(outputFilePath):
        comandocall = 'samtools' + ' view -q1 -b ' + sortedBamFile + ' | ' + 'bedtools' + ' intersect -abam stdin -b ' + intersectFile + ' -bed -wb -f 0.5 ' + '> ' + outputFilePath 
        print(comandocall)
        subprocess.call(
            comandocall, shell=True
        )
    else:
        print(outputFile, ' is already done')
    
    totalForBox = {}
    with open(outputFilePath, 'r') as intersectOut:
        intersectDf = pd.read_csv(
            intersectOut, sep='\t', usecols=[3, 12, 13, 14, 15],
            names=['intersected', 'chr', 'start', 'end', 'boxname'],
        )
        for index, ip in intersectDf.iterrows():
            intersectOcurrence = str(ip.intersected.split('/')[0])
            box = ','.join([str(ip.chr), str(ip.start), str(ip.end), ip.boxname])
            if box in totalForBox:
                totalForBox[box].add(intersectOcurrence)
            else:
                totalForBox[box] = {intersectOcurrence}

        for box in totalForBox:
            boxlen = len(totalForBox[box])
            totalForBox[box] = boxlen

        with open(outputFilePath[:-4] + '_boxtotals.csv', 'w') as elcsv:
            elcsv.write('chr,start,end,boxname,{}\n'.format(sortedBamFile.split('/')[-1][:-10]))
            for name, recount in totalForBox.items():
                elcsv.write('{},{}\n'.format(name, recount))

In [4]:
for fol in folders:
    performIntersect(fol,intersect_peaks_file_path)

/home/joaquin/projects/methylation/data/data_concentraciones/ZYMO/15/1ng/ZYMO15-round1_allpeaks_ZYMO151ng.bed
/home/joaquin/projects/methylation/data/data_concentraciones/ZYMO/15/1ng/ZYMO15-round1_allpeaks_ZYMO151ng.bed
samtools view -q1 -b /home/joaquin/projects/methylation/data/data_concentraciones/ZYMO/15/1ng/ZYMO151ngsorted.bam | bedtools intersect -abam stdin -b /home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/ZYMO15-round1_allpeaks.bed -bed -wb -f 0.5 > /home/joaquin/projects/methylation/data/data_concentraciones/ZYMO/15/1ng/ZYMO15-round1_allpeaks_ZYMO151ng.bed
/home/joaquin/projects/methylation/data/data_concentraciones/ZYMO/15/5ng/ZYMO15-round1_allpeaks_ZYMO155ng.bed
/home/joaquin/projects/methylation/data/data_concentraciones/ZYMO/15/5ng/ZYMO15-round1_allpeaks_ZYMO155ng.bed
samtools view -q1 -b /home/joaquin/projects/methylation/data/data_concentraciones/ZYMO/15/5ng/ZYMO155ngsorted.bam | bedtools intersect -abam stdin -b /home/joaquin/projects/m

In [7]:

def generateDfandNormaliceDataTPMs(folder,dataColName, totalReads):
    csvFile = glob.glob(f'{folder}/ZYMO15-round1_allpeaks*.csv')
    
    if len(csvFile) != 1:
        return print(folder, ' has a problem selecting File')
    else:
        csvFilePath = csvFile[0]

    fileDf = pd.read_csv(csvFilePath,header=0, names=['chr','star','end','id',dataColName])

    scalingFactor = totalReads/1000000
    fileDf_counts = fileDf.copy()
    fileDf[dataColName] = fileDf[dataColName].apply(lambda x: x/scalingFactor)

    return fileDf_counts, fileDf

In [6]:
path_to_total_reads = '/home/joaquin/projects/methylation/peaks_and_alingment/concentraciones_summary.tsv'
infosecund_df = pd.read_csv(path_to_total_reads,sep='\t',names=['condition','reads','q','peaks'])

In [8]:
main_dir = '/home/joaquin/projects/methylation/data/data_concentraciones/ZYMO/15'

listofdfs_tpms = []
listofdfs_counts = []

for workingFolder in folders:
    cachos = workingFolder.split('/')
    print(cachos)
    cachos[-3],cachos[-2],cachos[-1]
    datacolname = '{}_{}_{}'.format(cachos[-3],cachos[-2],cachos[-1])
    print(datacolname)
    normalization_value = int(list(infosecund_df[infosecund_df['condition'] == datacolname]['reads'])[0])
    print(normalization_value)
    counts, tpms = generateDfandNormaliceDataTPMs(workingFolder,datacolname,normalization_value)
    listofdfs_tpms.append(tpms)
    listofdfs_counts.append(counts)
    #             los valores que no estan en una de las replicas los completo con un 0
df_merged_tpms = reduce(lambda  left,right: pd.merge(left,right,on=['chr','star','end','id'],
                                how='outer'), listofdfs_tpms)

df_merged_counts = reduce(lambda  left,right: pd.merge(left,right,on=['chr','star','end','id'],
                                how='outer'), listofdfs_counts)




['', 'home', 'joaquin', 'projects', 'methylation', 'data', 'data_concentraciones', 'ZYMO', '15', '1ng']
ZYMO_15_1ng
9108005
['', 'home', 'joaquin', 'projects', 'methylation', 'data', 'data_concentraciones', 'ZYMO', '15', '5ng']
ZYMO_15_5ng
11313382
['', 'home', 'joaquin', 'projects', 'methylation', 'data', 'data_concentraciones', 'ZYMO', '15', '10ng']
ZYMO_15_10ng
10277543
['', 'home', 'joaquin', 'projects', 'methylation', 'data', 'data_concentraciones', 'ZYMO', '15', '25ng']
ZYMO_15_25ng
9319952
['', 'home', 'joaquin', 'projects', 'methylation', 'data', 'data_concentraciones', 'ZYMO', '15', '50ng']
ZYMO_15_50ng
9756143
['', 'home', 'joaquin', 'projects', 'methylation', 'data', 'data_concentraciones', 'ZYMO', '15', 'Input']
ZYMO_15_Input
8199725


In [9]:
df_merged_counts

,chr,star,end,id,ZYMO_15_1ng,ZYMO_15_5ng,ZYMO_15_10ng,ZYMO_15_25ng,ZYMO_15_50ng,ZYMO_15_Input
0,1,1440,1641,5ngP06598,33.0,96.0,71.0,51.0,71.0,8.0
1,1,9954,10155,5ngP17889,18.0,31.0,21.0,17.0,31.0,26.0
2,1,10737,10938,1ngP06790,89.0,87.0,49.0,40.0,53.0,12.0
3,1,11920,12121,1ngP08921,95.0,148.0,101.0,74.0,66.0,9.0
4,1,15423,15624,5ngP04555,59.0,170.0,90.0,79.0,103.0,11.0
...,...,...,...,...,...,...,...,...,...,...
26124,5,25332780,25332981,10ngP08014,NaN,22.0,10.0,3.0,5.0,5.0
26125,5,26073373,26073574,10ngP06143,NaN,4.0,2.0,1.0,8.0,4.0
26126,1,14824452,14824653,50ngP03977,NaN,NaN,1.0,2.0,1.0,7.0
26127,3,14893747,14893948,10ngP07858,NaN,NaN,1.0,NaN,1.0,4.0


In [10]:
df_merged_tpms

,chr,star,end,id,ZYMO_15_1ng,ZYMO_15_5ng,ZYMO_15_10ng,ZYMO_15_25ng,ZYMO_15_50ng,ZYMO_15_Input
0,1,1440,1641,5ngP06598,3.623186,8.485526,6.908266,5.472131,7.277466,0.975642
1,1,9954,10155,5ngP17889,1.976284,2.740118,2.043290,1.824044,3.177485,3.170838
2,1,10737,10938,1ngP06790,9.771624,7.690008,4.767676,4.291868,5.432475,1.463464
3,1,11920,12121,1ngP08921,10.430385,13.081853,9.827252,7.939955,6.764968,1.097598
4,1,15423,15624,5ngP04555,6.477818,15.026453,8.756957,8.476439,10.557451,1.341508
...,...,...,...,...,...,...,...,...,...,...
26124,5,25332780,25332981,10ngP08014,NaN,1.944600,0.972995,0.321890,0.512498,0.609777
26125,5,26073373,26073574,10ngP06143,NaN,0.353564,0.194599,0.107297,0.819996,0.487821
26126,1,14824452,14824653,50ngP03977,NaN,NaN,0.097300,0.214593,0.102500,0.853687
26127,3,14893747,14893948,10ngP07858,NaN,NaN,0.097300,NaN,0.102500,0.487821


In [11]:
df_merged_tpms.to_excel(f'{main_dir}/ZYMO15-round1_allpeaks_ZYMO15_tpms.xlsx', index=False)
df_merged_counts.to_excel(f'{main_dir}/ZYMO15-round1_allpeaks_ZYMO15_counts.xlsx', index=False)